Webscraper to get race results and add them to a postgresql database

In [1]:
import asyncio
from playwright.async_api import async_playwright

In [4]:
async def main():
    browsers = ['chromium', 'firefox', 'webkit']
    async with async_playwright() as p:
        for browser_type in browsers:
            browser = await p[browser_type].launch()
            page = await browser.new_page()
            await page.goto('https://amazon.com')
            await page.screenshot(path=f'py_{browser_type}.png', full_page=True)
            await page.wait_for_timeout(1000)
            await browser.close()
            
await main()

In [15]:
async with async_playwright() as p:
    browser = await p['chromium'].launch()
    page = await browser.new_page()
    await page.goto('https://track.rtrt.me/e/ATC-GRANDPRIX-1MILE-2025#/leaderboard/overall-sex-by-net-time-1mile-_ALL/FINISH')
    #await page.wait_for_load_state('leaderboardText prel whitebg leaderrow cpoint')
    await page.wait_for_timeout(5000)
    await page.screenshot(path=f'atc_test.png', full_page=True)
    spectator_button = page.get_by_text("I'm a Spectator")
    print(spectator_button.text_content)
    await spectator_button.click()
    #await page.screenshot(path=f'atc_test.png', full_page=True)
    await browser.close()
    
    

<bound method Locator.text_content of <Locator frame=<Frame name= url='https://track.rtrt.me/e/ATC-GRANDPRIX-1MILE-2025#/leaderboard/overall-sex-by-net-time-1mile-_ALL/FINISH'> selector='internal:text="I\'m a Spectator"i'>>


Future exception was never retrieved
future: <Future finished exception=TargetClosedError('Target page, context or browser has been closed')>
playwright._impl._errors.TargetClosedError: Target page, context or browser has been closed


TimeoutError: Locator.click: Timeout 30000ms exceeded.
Call log:
  - waiting for get_by_text("I'm a Spectator")


In [36]:
async def main():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=False) # headless=False lets you watch
        page = await browser.new_page()
        
        # Go to the page where the button will appear
        await page.goto("https://track.rtrt.me/e/ATC-GRANDPRIX-1MILE-2025#/leaderboard/overall-sex-by-net-time-1mile-_ALL/FINISH")
        
        print("Page loaded. Looking for the 'Spectator' button...")
        
        # Use a locator to find the button. Playwright will automatically wait
        # for it to appear before proceeding.
        
        
        #await page.pause()
        # Click the button
        await page.locator("iframe[name=\"rtframe\"]").content_frame.get_by_role("button", name="I'm a Spectator").click()
        
        print("Successfully clicked the 'Spectator' button.")
        
        # Now you can proceed to wait for your results and scrape them
        # For example, wait for a results container to be visible:
        frame_locator = page.frame_locator("iframe[name=\"rtframe\"]")
        rows = frame_locator.locator("tr.leaderrow")
        
        rows_count = await rows.count()
        print(f"Found {rows_count} rows.")
        all_rows_data = []
        for row in await rows.all():
            # For each row, get all cell locators
            cell_locators = row.locator("td")
            print('in loop')
            # Extract the text content from each cell and store it as a list
            row_data = [await cell.text_content() for cell in await cell_locators.all()]
            all_rows_data.append(row_data)
            
            # You can now process the data for the current row
            print(row_data)
        print("\n--- All Table Data ---")
        print(all_rows_data)

        # Add a small delay to see the result
        await page.wait_for_timeout(5000)
        
        await browser.close()

# To run this script:
await main()

Page loaded. Looking for the 'Spectator' button...
Successfully clicked the 'Spectator' button.
Found 0 rows.

--- All Table Data ---
[]
